In [49]:
import seaborn as sns
import pandas as pd
from ads_surkas.preprocessing import load_data, clean_data
import numpy as np
import math

In [50]:
# load data
# data = pd.read_csv("./.data/pakistan_ecommerce.csv", low_memory=False)
# data.loc[1048570]
data = clean_data(load_data("./.data/pakistan_ecommerce.csv"))

# data["Year"] = data["Year"].astype("int32")

# data["Month"] = data["Month"].astype("int32")
# print(data.dtypes)
#data["Time"] = pd.to_datetime(data["Year"]+data["Month"]+"1",format='%Y-%m-%d')
# data[(data["Year"] != 2016.0) & (data["Year"] != 2017.0) & (data["Year"] != 2018.0)]["Year"]
#data[["grand_total", "status"]].sort_values("grand_total").head(20)
data.head(20)


/home/ulrik2204/code/school/ads_surkas/ads_surkas/preprocessing/preprocess.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Year"] = data["Year"].astype("int32")
/home/ulrik2204/code/school/ads_surkas/ads_surkas/preprocessing/preprocess.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Month"] = data["Month"].astype("int32")
/home/ulrik2204/code/school/ads_surkas/ads_surkas/preprocessing/preprocess.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,item_id,status,created_at,sku,Customer ID,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,...,BI Status,Customer Since,M-Y,FY,price,qty_ordered,grand_total,Time,Year,Month
0,211131.0,complete,7/1/2016,kreations_YI 06-L,1.0,100147443,Women's Fashion,\N,0.0,cod,...,#REF!,2016-07-01,7-2016,FY17,1950.00,1.0,1950.00,2016-07-01,2016,7
1,211133.0,canceled,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,2.0,100147444,Beauty & Grooming,\N,0.0,cod,...,Gross,2016-07-01,7-2016,FY17,240.00,1.0,240.00,2016-07-01,2016,7
2,211134.0,canceled,7/1/2016,Ego_UP0017-999-MR0,3.0,100147445,Women's Fashion,\N,0.0,cod,...,Gross,2016-07-01,7-2016,FY17,2450.00,1.0,2450.00,2016-07-01,2016,7
3,211135.0,complete,7/1/2016,kcc_krone deal,4.0,100147446,Beauty & Grooming,R-FSD-52352,300.0,cod,...,Net,2016-07-01,7-2016,FY17,360.00,1.0,60.00,2016-07-01,2016,7
4,211136.0,order_refunded,7/1/2016,BK7010400AG,5.0,100147447,Soghaat,\N,0.0,cod,...,Valid,2016-07-01,7-2016,FY17,555.00,2.0,1110.00,2016-07-01,2016,7
5,211137.0,canceled,7/1/2016,UK_Namkino All In One 200 Gms,6.0,100147448,Soghaat,\N,0.0,cod,...,Gross,2016-07-01,7-2016,FY17,80.00,1.0,80.00,2016-07-01,2016,7
6,211138.0,complete,7/1/2016,kcc_krone deal,7.0,100147449,Beauty & Grooming,\N,300.0,cod,...,Net,2016-07-01,7-2016,FY17,360.00,1.0,60.00,2016-07-01,2016,7
7,211139.0,complete,7/1/2016,UK_Namkino Mix Nimco 400 Gms,6.0,100147450,Soghaat,\N,0.0,cod,...,Net,2016-07-01,7-2016,FY17,170.00,1.0,170.00,2016-07-01,2016,7
8,211140.0,canceled,7/1/2016,Apple iPhone 6S 64GB,8.0,100147451,Mobiles & Tablets,\N,0.0,ublcreditcard,...,Gross,2016-07-01,7-2016,FY17,96499.00,1.0,96499.00,2016-07-01,2016,7
9,211141.0,canceled,7/1/2016,Apple iPhone 6S 64GB,8.0,100147452,Mobiles & Tablets,\N,0.0,mygateway,...,Gross,2016-07-01,7-2016,FY17,96499.00,1.0,96499.00,2016-07-01,2016,7


In [51]:
data[data["Customer ID"] == 30465].tail(20)

,item_id,status,created_at,sku,Customer ID,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,...,BI Status,Customer Since,M-Y,FY,price,qty_ordered,grand_total,Time,Year,Month
511599,824355.0,complete,4/15/2018,MATSAM5A81910170A08,30465.0,100512334,Mobiles & Tablets,R SKZ 106980,5000.00,cod,...,Net,2017-01-01,4-2018,FY18,29960.0,1.0,24960.00,2018-04-01,2018,4
511606,824404.0,order_refunded,4/15/2018,MATNOK5A78229DE15BE,30465.0,100512372,Mobiles & Tablets,R SKZ 106980,1000.00,cod,...,Valid,2017-01-01,4-2018,FY18,6490.0,1.0,5490.00,2018-04-01,2018,4
511643,824449.0,order_refunded,4/15/2018,MATNOK5A78229DE15BE,30465.0,100512404,Mobiles & Tablets,R SKZ 106980,1000.00,cod,...,Valid,2017-01-01,4-2018,FY18,6490.0,1.0,5490.00,2018-04-01,2018,4
511676,824521.0,complete,4/15/2018,MATSAM5A7463EE3C1A5,30465.0,100512445,Mobiles & Tablets,R SKZ 106980,2000.00,cod,...,Net,2017-01-01,4-2018,FY18,16499.0,1.0,14499.00,2018-04-01,2018,4
511678,824562.0,order_refunded,4/15/2018,APPHAI59FDA03C5CCF0,30465.0,100512477,Appliances,R SKZ 106980,2000.00,cod,...,Valid,2017-01-01,4-2018,FY18,12315.0,1.0,10315.00,2018-04-01,2018,4
511727,824615.0,complete,4/15/2018,MATHUA5A731DFC8C5D7,30465.0,100512506,Mobiles & Tablets,R SKZ 106980,5000.00,cod,...,Net,2017-01-01,4-2018,FY18,29642.0,1.0,24642.00,2018-04-01,2018,4
511774,824666.0,received,4/15/2018,APPGAB5A617F11C9F29,30465.0,100512538,Appliances,R SKZ 106980,1000.00,cod,...,Valid,2017-01-01,4-2018,FY18,4520.0,1.0,4010.00,2018-04-01,2018,4
511775,824667.0,received,4/15/2018,DET59F31AB0BCEDF,30465.0,100512538,Superstore,R SKZ 106980,1000.00,cod,...,Valid,2017-01-01,4-2018,FY18,475.0,1.0,4010.00,2018-04-01,2018,4
511776,824668.0,received,4/15/2018,JAZ5A5F32D789415,30465.0,100512538,Superstore,R SKZ 106980,1000.00,cod,...,Valid,2017-01-01,4-2018,FY18,15.0,1.0,4010.00,2018-04-01,2018,4
512220,825367.0,complete,4/15/2018,APPDAW5A802A7DAD4DF,30465.0,100512974,Appliances,R SKZ 106980,10000.00,cod,...,Net,2017-01-01,4-2018,FY18,50999.0,1.0,40999.00,2018-04-01,2018,4


In [52]:
END_DATE = pd.to_datetime("2018-08-18")
ROWS = 115326
TILE = 28831


def label_customer(row, column, thresholds, labels):
    for i, th in enumerate(thresholds):
        min_th = -math.inf if i == 0 else thresholds[i - 1]
        if min_th < row[column] <= th:
            return labels[i]
    return labels[-1]


def label_customer_all_time(row):
    """
    Customer ID       69577.79
all_time_total     4198.00
order_count           1.00
Name: 0.63, dtype: float64
Customer ID       95825.76
all_time_total    20599.00
order_count           3.00
Name: 0.86, dtype: float64
Customer ID       108619.35
all_time_total     58792.00
order_count            7.00
Name: 0.95, dtype: float64
Customer ID       112601.6800
all_time_total    126729.9424
order_count           13.0000
Name: 0.98, dtype: float64
Customer ID       114517.53800
all_time_total    261230.69422
order_count           24.00000
Name: 0.993, dtype: float64
Customer ID       115052.83500
all_time_total    457120.35405
order_count           47.00000
Name: 0.9975, dtype: float64
    """
    thresholds = [4198.00, 20599.00, 58792.00, 126729.9424, 261230.69422, 457120.35405]
    labels = [
        "tiny",
        "small",
        "medium_small",
        "medium",
        "medium_large",
        "large",
        "very_large",
    ]
    return label_customer(row, "all_time_total", thresholds, labels)


# filter out canceled orders
data = data[
    (data["status"] == "complete")
    | (data["status"] == "received")
    | (data["status"] == "cod")
    | (data["status"] == "paid")
    | (data["status"] == "exchange")
    | (data["status"] == "payment_review")
    | (data["status"] == "pending")
    | (data["status"] == "processing")
    | (data["status"] == "pending_paypal")
]


customer_groups_all_time = (
    data.groupby("increment_id")
    .first()
    .groupby("Customer ID", as_index=False)
    .agg(
        all_time_total=("grand_total", "sum"),
        order_count=("grand_total", "count"),
        # customer_since=("Customer Since", "first"),
    )
)
# customer_groups_all_time["all_time_total_per_month"] = round(
#     customer_groups_all_time["all_time_total"]
#     / (
#         (END_DATE - customer_groups_all_time["customer_since"]) / np.timedelta64(1, "D")
#     ).astype(int)
#     * 30,
#     2,
# )

# customer_groups_all_time.sort_values("all_time_total").iloc[
#     math.floor(ROWS / 4) * 2 : math.floor(ROWS / 4 * 3)
# ]
customer_groups_all_time["group"] = customer_groups_all_time.apply(
    label_customer_all_time, axis=1
)
# customer_groups_all_time["group_month"] = customer_groups_all_time.apply(
#     label_customer_per_month, axis=1
# )
# a print(customer_groups_all_time)
# print(customer_groups_all_time.groupby("group")["Customer ID"].count())
# customer_groups_all_time.groupby("group")["all_time_total"].sum().reset_index().plot.bar(x="group")
# dat.to_csv(".data/all_time_total.csv")

# Finding groups of customers based on all_time_total
dat = (
    data.groupby("increment_id")
    .first()
    .groupby("Customer ID", as_index=False)
    .agg(all_time_total=("grand_total", "sum"), order_count=("grand_total", "count"))
)
print(dat.quantile(0.63)) #  7500.00
print(dat.quantile(0.86)) #   39717.33
print(dat.quantile(0.95)) #  114671.5875
print(dat.quantile(0.98)) #  265709.29
print(dat.quantile(0.993)) # 575067.925
print(dat.quantile(0.9975)) #  1323656

# print("Here starts all_time_total_per_month....")
# dat = customer_groups_all_time.groupby("Customer ID", as_index=False)[
#     "all_time_total_per_month"
# ].first()
# print(dat.quantile(0.63))  #  7500.00
# print(dat.quantile(0.86))  #   39717.33
# print(dat.quantile(0.95))  #  114671.5875
# print(dat.quantile(0.98))  #  265709.29
# print(dat.quantile(0.993))  # 575067.925
# print(dat.quantile(0.9975))  #  1323656
GROUPS = 4
TILE = 0
# customer_groups_all_time.sort_values("all_time_total_per_month").iloc[
#     math.floor(ROWS / GROUPS) * TILE : math.floor((ROWS / GROUPS) * (TILE + 1))
# ]
# customer_groups_all_time.groupby("group_month")[
#     "Customer ID"
# ].count()  # .plot.bar(x="group_month")
# customer_groups_all_time.groupby("group_month")[
#     "all_time_total"
# ].sum().reset_index().plot.bar(x="group_month")
customer_groups_all_time


Customer ID       69577.79
all_time_total     4198.00
order_count           1.00
Name: 0.63, dtype: float64
Customer ID       95825.76
all_time_total    20599.00
order_count           3.00
Name: 0.86, dtype: float64
Customer ID       108619.35
all_time_total     58792.00
order_count            7.00
Name: 0.95, dtype: float64
Customer ID       112601.6800
all_time_total    126729.9424
order_count           13.0000
Name: 0.98, dtype: float64
Customer ID       114517.53800
all_time_total    261230.69422
order_count           24.00000
Name: 0.993, dtype: float64
Customer ID       115052.83500
all_time_total    457120.35405
order_count           47.00000
Name: 0.9975, dtype: float64


,Customer ID,all_time_total,order_count,group
0,1.0,1950.00,1,tiny
1,3.0,1245.00,1,tiny
2,4.0,1282985.25,236,very_large
3,6.0,170.00,1,tiny
4,7.0,105.00,4,tiny
...,...,...,...,...
80029,115322.0,1298.00,1,tiny
80030,115323.0,44499.00,1,medium_small
80031,115324.0,549.00,1,tiny
80032,115325.0,1199.00,2,tiny


In [53]:
customer_total_per_month = (
    data.groupby(["increment_id"]).first().groupby(["Customer ID", "Time"], as_index=False).agg(monthly_total=("grand_total", "sum"))
    # data.groupby("Time", as_index=False).agg(monthly_total=("grand_total", "sum"), increment_id=("increment_id", "first"))
    # .groupby("Customer ID", as_index=False).agg(monthly_total=("grand_total", "sum"))
)
# customer_total_per_month.plot.bar()  #[["Customer ID", "Time", "grand_total"]]

def label_customer_per_month(row):
    # thresholds = [95.95, 342.365, 1588]
    """
    Customer ID      59837.99
monthly_total     2999.00
Name: 0.63, dtype: float64
Customer ID      90841.78
monthly_total    16000.00
Name: 0.86, dtype: float64
Customer ID      106434.7
monthly_total     47880.0
Name: 0.95, dtype: float64
Customer ID      111544.62
monthly_total     91733.96
Name: 0.98, dtype: float64
Customer ID      114087.189
monthly_total    172942.288
Name: 0.993, dtype: float64
Customer ID      114923.13500
monthly_total    267446.83215
Name: 0.9975, dtype: float64
    """
    thresholds = [2999.0, 16000.00, 47880.0, 91733.96, 172942.288, 267446.83215]
    labels = [
        "tiny_monthly",
        "small_monthly",
        "medium_small_monthly",
        "medium_monthly",
        "medium_large_monthly",
        "large_monthly",
        "very_large_monthly",
    ]
    return label_customer(row, "monthly_total", thresholds, labels)

customer_total_per_month["group_monthly"] = customer_total_per_month.apply(label_customer_per_month, axis=1)
# data.groupby("Customer ID").first().count()
# customer_total_per_month[customer_total_per_month["Customer ID"] == 5032]
# customer_total_per_month.groupby("Time", as_index=False).agg(group_count=("group_monthly", "count")).line(x="Time", y="group_count")
# customer_total_per_month.groupby("group_monthly")["Customer ID"].count().plot.pie()
# dat = customer_total_per_month
# print(dat.quantile(0.63))  #  7500.00
# print(dat.quantile(0.86))  #   39717.33
# print(dat.quantile(0.95))  #  114671.5875
# print(dat.quantile(0.98))  #  265709.29
# print(dat.quantile(0.993))  # 575067.925
# print(dat.quantile(0.9975))  #  1323656

In [54]:
# Add the customer categories to the data table
new_column = customer_groups_all_time.merge(customer_total_per_month, on="Customer ID")
data = pd.merge(data, new_column, on=["Customer ID", "Time"], how="left")
data.to_csv("./.data/pakistan_ecommerce_with_categories.csv")
# data[data["Customer ID"] == 5032] #[["Customer ID", "grand_total", "all_time_total", "monthly_total" "Customer Since"]]
# new_column[new_column["Customer ID"] == 5032]
# new_column
#print(data[["group"]])
data.groupby("Customer ID", as_index=False).first()# .groupby( "group", as_index=False).agg(count_of_distinct_customers=("all_time_total", "sum")).plot.bar( x="group", y="count_of_distinct_customers")  # .reset_index()["count_of_distinct_customers"].sum() #plot.bar(x="group_monthly", y="count_of_distinct_customers")
data.dtypes
new_column


,Customer ID,all_time_total,order_count,group,Time,monthly_total,group_monthly
0,1.0,1950.00,1,tiny,2016-07-01,1950.0,tiny_monthly
1,3.0,1245.00,1,tiny,2017-05-01,1245.0,tiny_monthly
2,4.0,1282985.25,236,very_large,2016-07-01,23655.0,medium_small_monthly
3,4.0,1282985.25,236,very_large,2016-08-01,69114.0,medium_monthly
4,4.0,1282985.25,236,very_large,2016-09-01,213937.9,large_monthly
...,...,...,...,...,...,...,...
114969,115322.0,1298.00,1,tiny,2018-08-01,1298.0,tiny_monthly
114970,115323.0,44499.00,1,medium_small,2018-08-01,44499.0,medium_small_monthly
114971,115324.0,549.00,1,tiny,2018-08-01,549.0,tiny_monthly
114972,115325.0,1199.00,2,tiny,2018-08-01,1199.0,tiny_monthly


In [55]:
# Categorize customers per all_time_total/customer_since
# customer_groups_per_month = customer_groups_all_time.copy()
# data["all_time_total_per_month"] = round(data["all_time_total"] / ((END_DATE - data["Customer Since"]) / np.timedelta64(1, "D")).astype(int) * 30, 2)
# data[["Customer ID", "all_time_total", "group", "increment_id", "Customer Since", "all_time_total_per_month"]]
# data.sort_values("all_time_total_per_month").groupby("increment_id").first().groupby("Customer ID", as_index=False).agg()
# data

In [56]:
data[data["Customer ID"] == 30465] #[["Customer ID", "increment_id", "grand_total", "price", "qty_ordered", "original_price", "discount_amount", "discount_percentage"]].to_csv("cust.csv")

,item_id,status,created_at,sku,Customer ID,increment_id,category_name_1,sales_commission_code,discount_amount,payment_method,...,qty_ordered,grand_total,Time,Year,Month,all_time_total,order_count,group,monthly_total,group_monthly
78744,382109.0,complete,1/5/2017,Infinix Note 3-2GB-Nationwide-Grey,30465.0,100257107,Mobiles & Tablets,R SKZ 106980,1000.00,cod,...,1.0,17599.00,2017-01-01,2017,1,1651566.651,130.0,very_large,17599.0,medium_small_monthly
134922,502761.0,received,6/1/2017,emart_0-34,30465.0,100320948,Appliances,R SKZ 106980,0.00,cod,...,1.0,1263.00,2017-06-01,2017,6,1651566.651,130.0,very_large,55945.0,medium_monthly
134923,502762.0,received,6/1/2017,e-seller_197,30465.0,100320948,Appliances,R SKZ 106980,0.00,cod,...,1.0,1263.00,2017-06-01,2017,6,1651566.651,130.0,very_large,55945.0,medium_monthly
137721,508648.0,complete,6/9/2017,ZA_Grand Prime Plus-Black,30465.0,100323787,Mobiles & Tablets,R SKZ 106980,3000.00,Payaxis,...,1.0,12399.00,2017-06-01,2017,6,1651566.651,130.0,very_large,55945.0,medium_monthly
137768,508781.0,complete,6/9/2017,LS_8961014003765,30465.0,100323854,Superstore,R SKZ 106980,280.00,Payaxis,...,2.0,1516.45,2017-06-01,2017,6,1651566.651,130.0,very_large,55945.0,medium_monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281569,826700.0,complete,4/15/2018,ENTSAM5A7BE6883BCB7,30465.0,100513675,Entertainment,R SKZ 106980,10000.00,cod,...,1.0,47291.00,2018-04-01,2018,4,1651566.651,130.0,very_large,254005.0,large_monthly
281570,826701.0,complete,4/15/2018,ENTSAM5A264AF81A12C,30465.0,100513675,Entertainment,R SKZ 106980,10000.00,cod,...,1.0,47291.00,2018-04-01,2018,4,1651566.651,130.0,very_large,254005.0,large_monthly
285685,835387.0,received,5/4/2018,APPHAI5ADC69ED243EF,30465.0,100518924,Appliances,R SKZ 106980,3930.55,jazzvoucher,...,1.0,37214.45,2018-05-01,2018,5,1651566.651,130.0,very_large,88370.1,medium_monthly
285710,835448.0,complete,5/4/2018,APPKEN5AD50F99A6F4A,30465.0,100518968,Appliances,R SKZ 106980,4755.94,jazzvoucher,...,1.0,45560.06,2018-05-01,2018,5,1651566.651,130.0,very_large,88370.1,medium_monthly


In [57]:
# Orders per time only (and their grand total)
data["item_total"] = data["price"] * data["qty_ordered"]
# data["discount_percentage"] = (data["item_total"] - data["grand_total"]) / data["item_total"]
orders = data.groupby("increment_id", as_index=False).agg(
    grand_total=("grand_total", "first"),
    Time=("Time", "first"),
    customer_id=("Customer ID", "first"),
    group=("group", "first"),
    group_monthly=("group_monthly", "first"),
    monthly_total=("monthly_total", "first"),
    all_time_total=("all_time_total", "first"),
    discount_amount=("discount_amount", "sum"),
    status=("status", "first"),
    qty_ordered=("qty_ordered", "sum"),
    customer_since=("Customer Since", "first"),
    payement_method=("payment_method", "first"),
    original_price=("item_total", "sum"),
    # mean_discount_percentage=("discount_percentage", "mean"),
)
orders["real_discount_amount"] = (orders["original_price"] - orders["grand_total"]).clip(
    lower=0
)
orders["discount_percentage"] = np.nan_to_num(
    orders["real_discount_amount"] / orders["original_price"], True, 0, 0
).clip(max=1)

orders.to_csv(".data/orders_per_time.csv")
# data[["discount_amount"]]
# data.head(20)
orders


,increment_id,grand_total,Time,customer_id,group,group_monthly,monthly_total,all_time_total,discount_amount,status,qty_ordered,customer_since,payement_method,original_price,real_discount_amount,discount_percentage
0,100001290,332.0,2016-11-01,19328.0,tiny,tiny_monthly,332.0,332.00,0.0,complete,1.0,2016-11-01,cod,332.0,0.0,0.000000
1,100147443,1950.0,2016-07-01,1.0,tiny,tiny_monthly,1950.0,1950.00,0.0,complete,1.0,2016-07-01,cod,1950.0,0.0,0.000000
2,100147446,60.0,2016-07-01,4.0,very_large,medium_small_monthly,23655.0,1282985.25,300.0,complete,1.0,2016-07-01,cod,360.0,300.0,0.833333
3,100147449,60.0,2016-07-01,7.0,tiny,tiny_monthly,60.0,105.00,300.0,complete,1.0,2016-07-01,cod,360.0,300.0,0.833333
4,100147450,170.0,2016-07-01,6.0,tiny,tiny_monthly,170.0,170.00,0.0,complete,1.0,2016-07-01,cod,170.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206579,100562383,550.0,2018-08-01,115325.0,tiny,tiny_monthly,1199.0,1199.00,0.0,cod,1.0,2018-08-01,cod,400.0,0.0,0.000000
206580,100562384,649.0,2018-08-01,115325.0,tiny,tiny_monthly,1199.0,1199.00,0.0,cod,1.0,2018-08-01,cod,499.0,0.0,0.000000
206581,100562385,849.0,2018-08-01,115320.0,tiny,tiny_monthly,849.0,849.00,0.0,cod,1.0,2018-08-01,cod,699.0,0.0,0.000000
206582,100562386,35899.0,2018-08-01,115326.0,medium_small,medium_small_monthly,35899.0,35899.00,0.0,processing,1.0,2018-08-01,bankalfalah,35599.0,0.0,0.000000


In [58]:
# # Total Revenue in November 2016
# total = 0
# for i, row in data.groupby("increment_id").first().reset_index().iterrows():
#     order_grand_total = row["grand_total"]
#     y = row["Time"].year
#     m = row["Time"].month
#     if (m == 11 and y == 2016):
#         total += order_grand_total
# print("total: ", total)

In [59]:
# total1 = 0
# total2 = 0
# total3 = 0
# checked = []
# for i, row in data.iterrows():
#     if row["Customer ID"] == 5032 and row["increment_id"] not in checked:
#         total2 += row["grand_total"]
#         checked.append(row["increment_id"])
#     if row["Customer ID"] == 5032: 
#         total1 += row["price"] * row["qty_ordered"]
#         total3 += row["price"] * row["qty_ordered"] - row["discount_amount"]
#         checked.append(row["increment_id"])

In [60]:
# print("price * qty", total1)
# print("sum of grand total", total2)
# print("price * qty - discount", total3)

In [61]:
# dat = data.groupby("increment_id").first().groupby("Customer ID", as_index=False).agg(all_time_total=("grand_total","sum"), order_count=("grand_total", "count"))
# dat = data.copy() #.groupby("Customer ID", as_index=False).agg(all_time_total=("grand_total","sum"), order_count=("grand_total", "count"))
# dat["all_time_price"] = dat["price"] * dat["qty_ordered"] - dat["discount_amount"]
# dat = dat.filter(["Customer ID", "increment_id", "price", "qty_ordered", "discount_amount", "grand_total", "all_time_price"])
# cust = dat.groupby("Customer ID", as_index=False).agg(total_price=("all_time_price", "sum"))
# all = cust.merge(dat, on="Customer ID")
# dat[dat["Customer ID"] == 5032]

# dat[dat["Customer ID"] == 5032].tail(20)

In [62]:
# g = sns.PairGrid(data, vars=['category_name_1', 'grand_total', 'Time'])
# g.map_upper(sns.histplot)